In [16]:
from sqlalchemy import create_engine, Table, MetaData, select 
hostname = 'localhost'
username = 'root'
password = 'Bautroixanh12345'
database_name = 'web_scrapping'
connection_url = f'mysql://{username}:{password}@{hostname}/{database_name}'
engine = create_engine(url=connection_url)
table_name1 = 'colors_backup'
table_name2 = 'color_hunt'
table_name3 = 'color_drop'
table_name4 = 'unsplash'
metadata = MetaData()
table1 = Table(table_name1, metadata, autoload_with=engine)
table2 = Table(table_name2, metadata, autoload_with=engine)
table3 = Table(table_name3, metadata, autoload_with=engine)
table4 = Table(table_name4, metadata, autoload_with=engine)
select_statement1 = select(table1)
select_statement2 = select(table2)
select_statement3 = select(table3)
select_statement4 = select(table4)
with engine.connect() as connection: 
    results1 = connection.execute(select_statement1)
    rows1 = results1.fetchall()

    results2 = connection.execute(select_statement2)
    rows2 = results2.fetchall()

    results3 = connection.execute(select_statement3)
    rows3 = results3.fetchall()

    results4 = connection.execute(select_statement4)
    rows4 = results4.fetchall()

In [17]:
import pandas as pd 
import numpy as np 
df1 = pd.DataFrame(rows1)
df2 = pd.DataFrame(rows2)
df3 = pd.DataFrame(rows3)
df4 = pd.DataFrame(rows4)

In [18]:
df4.drop(columns='image_name', inplace=True)

In [19]:
def ranking_column(data_frame): 
    column_list = data_frame.columns.tolist()
    column_list = [column.lower() for column in column_list]
    if 'score' in column_list: 
        quantile = [0, 0.25, 0.5, 0.75, 1]
        labels = [0, 1, 2, 3]
        data_frame['score'] = pd.to_numeric(data_frame['score'])
        data_frame.sort_values(by='score', ascending=True, inplace=True)
        data_frame.insert(len(column_list), 'rank', pd.qcut(data_frame['score'], q=quantile, labels=labels))
    else: 
        print(f"Can't conduct ranking for {data_frame}, no column score found")
    return data_frame
df2 = ranking_column(df2)
df3 = ranking_column(df3) 
df4 = ranking_column(df4)  

In [20]:
machine_learning_df = pd.concat([df1, df2, df3, df4], ignore_index=True)
machine_learning_df.drop(columns='id', inplace=True)

In [21]:
machine_learning_df['rank'] = pd.to_numeric(machine_learning_df['rank'])

In [22]:
input_features = machine_learning_df['color_code'].tolist()
targeted_variable = machine_learning_df['rank'].tolist() 
vocabulary_size = len(machine_learning_df['color_code'].unique())

In [23]:
from sklearn.preprocessing import LabelEncoder
encoded_dict = {}
label_encoder = LabelEncoder() 
encoded_color_codes = label_encoder.fit_transform(input_features)
for color, encoded_color in zip(input_features, encoded_color_codes): 
    encoded_dict[color] = encoded_color 
machine_learning_df['encoded_color_code'] = machine_learning_df['color_code'].map(encoded_dict)

In [24]:
unique_feature = machine_learning_df['encoded_color_code'].unique().tolist()
print(f'There is the total of {len(unique_feature)} unique features out of {len(machine_learning_df)} features')

There is the total of 24812 unique features out of 77080 features


In [25]:
def hex_to_rgb(hex_color):
    # Remove '#' if present
    hex_color = hex_color.lstrip('#')
    # Convert hex to RGB
    rgb = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return rgb

color_code_list = machine_learning_df['color_code'].tolist()
output_list = [] 
for code in color_code_list:
    output = hex_to_rgb(code)
    output_list.append(output)

red_list = [] 
green_list = [] 
blue_list = [] 
for rgb_code in output_list: 
    red_channel = rgb_code[0]
    green_channel = rgb_code[1]
    blue_channel = rgb_code[2]
    red_list.append(red_channel)
    green_list.append(green_channel)
    blue_list.append(blue_channel)

In [26]:
machine_learning_df['red_channel'] = red_list 
machine_learning_df['green_channel'] = green_list 
machine_learning_df['blue_channel'] = blue_list
machine_learning_df['red_channel'] = pd.to_numeric(machine_learning_df['red_channel'])
machine_learning_df['green_channel'] = pd.to_numeric(machine_learning_df['green_channel'])
machine_learning_df['blue_channel'] = pd.to_numeric(machine_learning_df['blue_channel'])
features_list = ['encoded_color_code', 'red_channel', 'green_channel', 'blue_channel']
input_features = machine_learning_df[features_list].values.tolist()

In [27]:
from sqlalchemy import inspect 
inspector = inspect(engine) 
user = input('Updating the machine learning table?')
if user.lower().startswith('y'): 
    print('Processing the input table...')
    while True: 
        table_name = input('Enter the table name\n press "exit" to escape the function')
        if inspector.has_table(table_name): 
            machine_learning_df.to_sql(table_name, con=engine, index=False, if_exists='replace')
            print(f'The total rows {len(machine_learning_df)} has been updated')
            break 
        elif table_name.lower().startswith('e'):
            print('Escaping the function')
            break   
        elif table_name.lower() == 'unsplash': 
            print('Attempting to access the wrong table')
            break 
        else: 
            print(f'The table {table_name} was not found')  
elif user.lower().startswith('n'): 
    print('Exiting the current function')

Processing the input table...
The total rows 77080 has been updated


In [38]:
import torch 
from torch import nn 
from torch.utils.data import TensorDataset, DataLoader 
import numpy as np 

#Expecting the data to have around 160k to 240k total data samples for this feature

proportion = int(0.8*len(input_features))
x_train, y_train = input_features[:proportion], targeted_variable[:proportion]
x_test, y_test = input_features[proportion:], targeted_variable[proportion:]

x_train, y_train = np.array(x_train), np.array(y_train)
x_test, y_test = np.array(x_test), np.array(y_test)

x_train, y_train = torch.from_numpy(x_train).type(torch.float32), torch.from_numpy(y_train).type(torch.float32)
x_test, y_test = torch.from_numpy(x_test).type(torch.float32), torch.from_numpy(y_test).type(torch.float32)

BATCH_SIZE = 128
train_dataset = TensorDataset(x_train, y_train)
train_dataloader = DataLoader(dataset=train_dataset, 
                              shuffle=True,
                              batch_size=BATCH_SIZE)
test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(dataset=test_dataset, 
                             shuffle=False, 
                             batch_size=BATCH_SIZE)
#Create the neural network 
class ColorCode(nn.Module): 
    def __init__(self, 
                 hidden_size, 
                 num_layers, 
                 embedded_size,
                 vocabulary_size, 
                 dropout_rate,  
                 output_size): 
        super(ColorCode, self).__init__() 
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedded_size = embedded_size
        self.dropout_rate = dropout_rate
        self.embedding = nn.Embedding(num_embeddings=vocabulary_size, embedding_dim=embedded_size)
        self.lstm = nn.LSTM(input_size=embedded_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.stacked_layer = nn.Sequential(
            nn.Linear(in_features=hidden_size, out_features=1026), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(in_features=1026, out_features=526), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(in_features=526, out_features=256), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate),
            nn.Linear(in_features=256, out_features=126), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(in_features=126, out_features=64), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(in_features=64, out_features=32), 
            nn.ReLU(), 
            nn.Dropout(dropout_rate), 
            nn.Linear(in_features=32, out_features=output_size)
        )
    
    def forward(self, x): 
        batch_size = x.size(0)
        x = x.type(torch.LongTensor)
        embedded_out = self.embedding(x)
        #Output size of the embedded layer => (batch_size, sequence_length, output_dim)
        embedded_out = embedded_out.transpose(1, 2)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        lstm_out, _ = self.lstm(embedded_out, (c0, h0))
        lstm_out = lstm_out[:, -1, :]
        output = self.stacked_layer(lstm_out)
        return output 
#Initailize the model 
model = ColorCode(num_layers=1, hidden_size=16, vocabulary_size=vocabulary_size+1, embedded_size=4, output_size=4, dropout_rate=0.4)
loss_fn = nn.CrossEntropyLoss() 
def acc_fn(y_pred, y_true): 
    correct = torch.eq(y_pred, y_true).sum().item() 
    acc = (correct / len(y_pred)) * 100 
    return acc 
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#Create the training and testing loop
epoches = 10
for epoch in range(epoches):
    print(f'Epoch {epoch}\n-----') 
    model.train() 
    for batch, (train_features, train_labels) in enumerate(train_dataloader):
        train_pred = model(train_features)
        train_loss = loss_fn(train_pred, train_labels.type(torch.long))
        train_loss.backward() 
        optimizer.step() 
        optimizer.zero_grad() 
        if batch % 400 == 0: 
            print(f'Looked at {batch * len(train_features)}/{len(train_dataloader.dataset)} samples')
        
    with torch.inference_mode(): 
        model.eval() 
        for test_features, test_labels in test_dataloader: 
            test_pred = model(test_features)
            test_loss = loss_fn(test_pred, test_labels.type(torch.long))
            test_acc = acc_fn(torch.argmax(torch.softmax(test_pred, dim=1), dim=1), test_labels)
        print(f'Train loss {train_loss.item():.4f} | Test loss {test_loss.item():.4f} | Test acc {test_acc}')

Epoch 0
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.2509 | Test loss 2.6262 | Test acc 0.0
Epoch 1
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.3099 | Test loss 2.6246 | Test acc 0.0
Epoch 2
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.3124 | Test loss 2.6778 | Test acc 0.0
Epoch 3
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.2298 | Test loss 2.6597 | Test acc 0.0
Epoch 4
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.2460 | Test loss 2.6779 | Test acc 0.0
Epoch 5
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.3065 | Test loss 2.6705 | Test acc 0.0
Epoch 6
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.2238 | Test loss 2.6551 | Test acc 0.0
Epoch 7
-----
Looked at 0/61664 samples
Looked at 51200/61664 samples
Train loss 1.2782 | Test loss 2.6529 | Test acc 0.0
Epoch 8
-----
Looked at 

In [ ]:
machine_column = machine_learning_df.columns.tolist()
machine_learning_df.drop_duplicates(subset=machine_column, keep='last', ignore_index=True, inplace=True)
len(machine_learning_df)

62891

In [ ]:
import pandas as pd 
continue_df6 = pd.read_excel('continue_working6.xlsx', names=['image_name', 'score'])

In [ ]:
from color_normalizing import ColorCode, SubCode 
folder_name = 'continue_working6.xlsx'
sub_code_output = SubCode.dataframe_converting(folder_name)

In [ ]:
import importlib 
import color_normalizing
importlib.reload(color_normalizing)

<module 'color_normalizing' from 'd:\\Machine Learning\\Day 5 - LSTM\\color_normalizing.py'>

In [35]:
machine_learning_df['rank'].value_counts()

rank
0    19292
1    19270
3    19264
2    19254
Name: count, dtype: int64